In [1]:
#Fresh Cortex from Adult Mouse Brain (P50) - Epi ATAC v.10 - fresh
 

In [2]:
#install packages
#!pip install pandas
#!pip install scipy
#!pip install pyfaidx
#!pip install scikit-learn 
import sys
print(sys.executable)

/opt/anaconda3/envs/VIB_model/bin/python


In [2]:
import pandas as pd
from scipy.io import mmread
from scipy.sparse import csr_matrix
import numpy as np 

#load matrix
matrix = mmread("filtered_peak_bc_matrix/matrix.mtx").tocsc()

#load peaks
peaks = pd.read_csv("filtered_peak_bc_matrix/peaks.bed", sep = "\t", header = None)
barcodes = pd.read_csv("filtered_peak_bc_matrix/barcodes.tsv", header = None)

#assign rows and cols
peaks.columns = ["chr", "start", "end"]
barcodes.columns = ["cell_barcode"]

In [3]:
matrix_binary = (matrix>0).astype(int)
#sup per cell - how many cells have that peak accessible /peak cessible per cell - cols are cells but are not shown
peak_accessibility = matrix_binary.sum(axis=1)

print(peak_accessibility)
#convert to A1 numpy array
peaks_accessibility_1D = peak_accessibility.A1

[[260]
 [731]
 [ 86]
 ...
 [453]
 [407]
 [ 61]]


In [4]:
from pyfaidx import Fasta
#load genome  Fasta file
genome = Fasta("Genomes/Mus_musculus.GRCm39.dna.primary_assembly.fa")

#replace chr as str name because in genome there is only number
peaks['chr'] = peaks['chr'].str.replace('chr',"")
#function to get sequence
def get_sequence(row):
    return genome[row['chr']][row['start']:row['end']].seq

peaks['sequence'] = peaks.apply(get_sequence, axis=1) #aply the funcition getsequences



In [6]:
#Pad if seq > 150bp or trim it if is is larger
def pad_or_trim(seq, window_size = 150):
    seq = seq.upper()
    if len(seq) > window_size:
        return seq[:window_size]
    return seq + "N" * (window_size - len(seq)) #Pad with Ns


#Create one hot encoding function for the sequence
def one_hot_encoding(sequence):
    mapping = {"A":0, "C":1, "G":2, "T":3}
    one_hot = np.zeros((len(sequence), 4), dtype = np.int8) #rows-len of seq, and cols 4 possible nts
    for i, nt in enumerate(sequence.upper()):
        if nt in mapping:
            one_hot[i, mapping[nt]] = 1
    return one_hot


#add trimmed seq in the peaks 
peaks['trimmed_sequence'] = peaks['sequence'].apply(pad_or_trim)

#for all seqs
encoded_sequences = [one_hot_encoding(seq) for seq in peaks['trimmed_sequence']]


In [13]:
#Define a CNN model 

X = np.array(encoded_sequences)
print(X.shape)
import tensorflow as tf
from tensorflow.keras import layers, models
Y = peaks_accessibility_1D
#perfrom standarization - difffernt scales in values (10-100 and 10K-20K), so correct for that 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Y = scaler.fit_transform(peaks_accessibility_1D.reshape(-1, 1)).flatten()#-1,1 makes 2 D array bcs scalar expects 2D->flatten back again


(157797, 150, 4)


In [24]:
"""Split the Data"""
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models, metrics
X_train , X_test , Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=42) #20% in test data set, 80%train, random_state ensure reprodcibility

#Starting a simple CNN model

model = models.Sequential([
    layers.Conv1D(64,10, activation = "relu", input_shape = (150,4)),
    layers.MaxPooling1D(3),
    layers.Conv1D(128, 10, activation = "relu"),
    layers.GlobalMaxPooling1D(),
    layers.Dense(64, activation = "relu"),
    layers.Dense(1, activation = "linear")
])
    
model.compile(optimizer = 'adam',
              loss = 'mean_squared_error',
              metrics = [
                  metrics.MeanAbsoluteError(name= "mae"),
                  metrics.MeanSquaredError(name = "mse"),
                  metrics.RootMeanSquaredError(name = "rmse")
              ])
model.summary()


#Train the model
model.fit(X_train, Y_train, epochs = 30, batch_size=32, validation_data = (X_test, Y_test))



Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_16 (Conv1D)              │ (None, 141, 64)        │         2,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_8 (MaxPooling1D)  │ (None, 47, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_17 (Conv1D)              │ (None, 38, 128)        │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_7          │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 92,993 (363.25 KB)

 Trainable params: 92,993 (363.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
3945/3945 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - loss: 0.9934 - mae: 0.6324 - mse: 0.9934 - rmse: 0.9967 - val_loss: 0.9404 - val_mae: 0.5970 - val_mse: 0.9404 - val_rmse: 0.9698
Epoch 2/30
3945/3945 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - loss: 0.9772 - mae: 0.6212 - mse: 0.9772 - rmse: 0.9885 - val_loss: 0.9396 - val_mae: 0.5978 - val_mse: 0.9396 - val_rmse: 0.9693
Epoch 3/30
3945/3945 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - loss: 0.9816 - mae: 0.6213 - mse: 0.9816 - rmse: 0.9907 - val_loss: 0.9372 - val_mae: 0.6116 - val_mse: 0.9372 - val_rmse: 0.9681
Epoch 4/30
3945/3945 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - loss: 0.9580 - mae: 0.6125 - mse: 0.9580 - rmse: 0.9784 - val_loss: 0.9387 - val_mae: 0.6308 - val_mse: 0.9387 - val_rmse: 0.9689
Epoch 5/30
3945/3945 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - loss: 0.9775 - mae: 0.6194 - mse: 0.9775 - rmse: 0.9886 - val_loss: 0.9379 - val_mae: 0.6246 - val_mse: 0.9379 - val_rmse: 0.9685
Epoch 6/30
3945/3945 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - loss: 0.